First we will need a package manager to load a variety of functions that we will need further down the code.
This package manager is called "Pkg", we launch it by using the command "using Pkg"
Then we load all of the packages necessary for the algorithm.
- We need "IJulia" to be able to use Julia in this Jupyter Notebook.
- XLSX will allow us to load data from our excel file into the code.
- OSCAR (Open Source Computer Algebra Research) is a powerful computer algebra system which will allow us to build and handle polynomial rings and other algebraic structures.
We launch OSCAR by calling the command "using Oscar".


In [3]:
using Pkg
Pkg.add("IJulia")
Pkg.add("XLSX")
Pkg.add("Oscar")
import XLSX
using Oscar

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.11.3 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


Next, we need to build our algebraic structure, i.e., our Polynomial Ring.
We map the variable R1 to an object of abstract type residue ring of integers modulo 2, i.e. {0,1}.
Then, we construct a multivariate polynomial ring myR in (A, B, C, D, E, F, G) over integers.
The vector of strings ["MH", "L", "P", "R", "D", "PD", "LD"] as argument of the PolynomialRing constructor defines how our generating values A, B, C, D, E, F, G should be printed. We will later select the object properties matching these strings (i.e. "M" = missing head; "L" = limestone; ...) from the excel sheet.
We assign the list of variables in myR to a variable myvars.


In [4]:
R1 = ResidueRing(ZZ, 2);
myR, (A, B, C, D, E, F, G) = PolynomialRing(R1, ["MH", "L", "P", "R", "D", "PD", "LD"]);
myvars=[A, B, C, D, E, F, G];

From the excel file named "Cachette_Matrix_fuer_Noor.xlsx", we extract the information about each object in the Cachette. By information we mean, that for each object, we get a list of values in {0,1}, with 0 meaning the object does not have a specific object property and "1" meaning it does.
The variable chartable is thus in a form of an 333x16 array, for 333 objects and their {0,1} value for 16 object propertiesWe assign the variable "indices" to the numbers 1 through 333 to be able to access each object by it's position in the table.
i.e., with indice 1, we access the first object, with indice 2 the second and so on...

In [5]:
chartable=XLSX.readdata("Cachette_Matrix_fuer_Noor.xlsx", "Sheet1", "A2:P334");
indices=1:333;
#this function helps us find any statues with the numerical encoding "1" for the object propoerties 
#coded as their column number in the excel sheet
obj_properties = Any[7,11,16]
rowcount = 0
list = []
for row in eachrow(chartable)
    verification_list = []
    rowcount += 1
    for property in obj_properties
        if row[property] == 1;
            push!(verification_list, 1)
        else
            push!(verification_list, 0)
        end
    end
    if 0 ∉ verification_list;
        push!(list, rowcount)
    end      
end 

println(list)


Any[11]


Now we want to select from those object properties, the 7 properties that should be the generating objects of our polynomial ring. Later on we can change which object properties we want to use by switching out the integers in  "choice".
__varnum__ refers to the amount of properties (i.e. the columns in the excel sheet) we select.
__choice__ is an array of the integers refering to the columns in our excel sheet/chartable we want to select.
1 will select column 1 of the chartable, referring to the object property in the first column of the excel sheet
3 will select column 3 in the chartable, referring to the object property in the 3rd coloumn of the excel sheet
etc.

In [11]:
varnum=7; choice=[4,11,14,15,16,6,7]; 

We reassign **chartable** to the selection of columns according to our selection in "choice". We thus get a 333:7 array of objects and their {0,1} charactarization according to the 7 selected object properties (which here are column 4, 11, 14, 15, 16, 6 and 7 but can be switched out to analyze patterns and rules in a different set of object properties).
We start building a list of object polynomials **expr_list** for each object i. chartable[i,:] selects the list of 0,1 xistics for object i.<br>
e.g.,
for the first object the following operation is performed to get its polynomial expression:<br>
prod([A,B,C,D,E,F,G]+ [A,B,C,D,E,F,G]^0 - [0,0,1,1,0,0,1]) = prod ([A,B,C,D,E,F,G]+[1,1,1,1,1,1,1]-[0,0,1,1,0,0,1]) <br>
                                                     = (A+1) * (B+1) * (C+1-1) * (D+1-1) * (E+1) * (F+1) * (G+1-1) <br>
                                                     = (A+1) * (B+1) * C * D * (E+1) * (F+1) * G <br>
We then iterate through the remainder of the objects and extend **expr_list** to get a list of all object polynomials.</p>

In [ ]:
chartable=chartable[:,choice];
expr_list=prod(myvars+ myvars.^0 -chartable[1,:]); 
for i=2:size(chartable,1)
    expr_list=vcat(expr_list, prod(myvars+ myvars.^0 -chartable[i,:])); 
end
expr_list;

We then compute the weights of the object properties. (we will need them later in the generation of the Gröbner basis)
We first create an array of ones of length 7 (varnum). We then iterate through the list of properties to print out the sum of objects that do have the property ("true="), - i.e. the value for that object in the column of said object property is 1 - and the sum of objects that do not have the property ("false=").
Finally, we calculate the weight for the specific object property by multiplying the sum of the values in the respective column by the difference between the amount of objects in total and the sum of the values in the respective column)
i.e., <br>
let sum_i = sum of values (0 or 1) for all the objects of property i <br>
    nr_of_objects = number of objects on chartable <br>
then the weight of object property i = sum_i * (nr_of_objects - sum_i)

We then add each weight to the array of ones **monweight** that we created


In [14]:
monweight=ones(Int,varnum);
for i=1:size(myvars,1)
    print(myvars[i]);
    print(" true=");
    print(sum(chartable[:,i]));
    print(" false=");
    print(size(chartable,1)-sum(chartable[:,i]));
    print("\n");
    monweight[i]=sum(chartable[:,i])*(size(chartable,1)-sum(chartable[:,i]));
end
monweight

MH true=45 false=288
L true=97 false=236
P true=264 false=69
R true=57 false=276
D true=22 false=311
PD true=104 false=229
LD true=18 false=315


7-element Vector{Int64}:
 12960
 22892
 18216
 15732
  6842
 23816
  5670

We now want to get the union of all polynomial expressions. Normally the union of two objects A, B is computed by A+B+(A\*B).
However, since A\*A = 0 and A+A = A; instead of creating the union of all polynomial expressions, we can also find the list of all **unique** polynomial expressions first, then just add them and get the same result. 
In that light, from our list of expressions of our objects, we now want to generate the list of unique expressions, **uq_expr_list**.
We then add these unique expressions (sum(uq_expr_list)). This union includes everything that has been observed.
To get the expression that includes everything that has **not** been observed, we need to take the complement of said union, by adding "1". <br>
(1 here is equal to the union of A,B,C,D,E,F,G))
We assign the result to variable **expr**.<br>
So far, we have created a residual ring but it still lacks the boolean property.
We begin building our generator by including all expressions that describe the boolean property, i.e. idempotency:<br>
In "myvars.^2+myvars", we generate all expressions (A\*A)+A (because if (A\*A)+A is in the ideal, then (A\*A)+A = 0 and therefore A\*A = A which is idempotency). We assign these expressions to the variable **generator** <br>
We then build our ideal **II**, of myR generated by *vcat(generator, expr)*, which is the union of everything that has not been observed and all the expressions of idempotency.<br>

quo(myR, II) will return the quotient Y of the module myR by the module II and the canonical quotient map from myR to Y.<br>

We then construct the Gröbner basis of the ideal II with respect to the ordering defined by "wdeglex((gens(myR), monweight)"<br>
wdeglex((gens(myR), monweight) returns the corresponding weighted lexicographical ordering on the set of monomials in the variables of myR.


In [15]:
uq_expr_list=unique(expr_list);
expr=sum(uq_expr_list)+1;
generator=myvars.^2+myvars; II=ideal(myR, vcat(generator, expr));
Y, m = quo(myR, II);
GB = groebner_basis(II, ordering = wdeglex(gens(myR),monweight))

Gröbner basis with elements
1 -> LD^2 + LD
2 -> D^2 + D
3 -> P*LD + R*LD + D*LD + LD
4 -> P*D
5 -> MH*D*LD + D*LD
6 -> MH^2 + MH
7 -> R*D*LD
8 -> R^2 + R
9 -> P*R + R*D + P + R + D + 1
10 -> L*D*LD + D*LD
11 -> MH*R*D + MH*P + MH*R + MH*D + MH
12 -> D*PD*LD
13 -> P^2 + P
14 -> MH*L*LD + L*D*LD + MH*R*LD + R*D*LD + MH*LD + D*LD
15 -> MH*PD*LD + D*PD*LD + MH*R*LD + R*D*LD
16 -> L*R*D + L*P + L*R + L*D + L
17 -> L^2 + L
18 -> PD^2 + PD
19 -> L*PD*LD + L*D*LD + D*LD
20 -> MH*P*PD + MH*L*P + MH*R*PD + MH*L*R + MH*D*PD + MH*L*D + MH*PD + MH*L + MH*P + MH*R + MH*D + MH
21 -> L*P*PD + L*R*PD + L*D*PD + L*PD + P*D
22 -> MH*L*D*PD + MH*L*P + MH*L*R + MH*D*PD + MH*L*D + MH*L + MH*P + MH*R + MH*D + MH
23 -> MH*L*R*PD + MH*L*D*PD + L*P*PD + L*R*PD + P*R*PD + L*D*PD + MH*L*R + L*P*D + L*PD + R*D*PD + L*R*D + MH*D*PD + P*PD + L*P + R*PD + L*R + P*R + MH*P + D*PD + L*D + MH*R + P*D + PD + L + R*D + MH*D + P + R + MH + D + 1
with respect to the ordering
wdeglex([MH, L, P, R, D, PD, LD], [12960, 22892, 

Here we are printing out what new rules we would find if we considered each object (polynomial) as an exception.

In [15]:
for i = 1:size(uq_expr_list,1)
    res=normal_form(uq_expr_list[i],gens(GB));
    counter=0;
    for j = 1:size(expr_list,1)
        if (uq_expr_list[i]==expr_list[j])
            counter=counter+1;
            printstyled(indices[j], color=:red);
            print(" ");
        end
    end
    printstyled(" (#", color=:red);
    printstyled(counter, color=:red);
    printstyled(")", color=:red);
    print("\n");
    printstyled(factor(uq_expr_list[i]), color=:green);
    wrtnorm=0;
    if (wrtnorm==1)
        printstyled("=\n", color=:green);
        printstyled(uq_expr_list[i], color=:green);
    end
    print("\n");
    printstyled(res, bold=:true);
    print("\n\n");
end

1  (#1)
1 * D * PD * (LD + 1) * (R + 1) * (P + 1) * (L + 1) * (MH + 1)
MH*L*R*D + MH*R*D*PD + MH*R*D + L*D*PD + R*D*PD + D*PD

2 12  (#2)
1 * MH * D * (LD + 1) * (PD + 1) * (R + 1) * (P + 1) * (L + 1)
MH*L*D + MH*R*D*PD + MH*D + L*R*D*PD

3 4 5 8  (#4)
1 * L * D * PD * (LD + 1) * (R + 1) * (P + 1) * (MH + 1)
MH*L*R*D + MH*R*D + MH*D*PD + L*D*PD

6  (#1)
1 * MH * L * D * (LD + 1) * (PD + 1) * (R + 1) * (P + 1)
MH*L*D + MH*R*D + MH*D*PD + L*R*D*PD + D*LD

7  (#1)
1 * MH * L * D * PD * (LD + 1) * (R + 1) * (P + 1)
MH*L*R*D + MH*R*D + MH*D*PD + L*R*D*PD

9 10  (#2)
1 * L * D * (LD + 1) * (PD + 1) * (R + 1) * (P + 1) * (MH + 1)
MH*L*D + MH*R*D + MH*D*PD + L*R*D + L*D*PD + L*D

11  (#1)
1 * MH * L * D * LD * (PD + 1) * (R + 1) * (P + 1)
D*LD

13 14 15  (#3)
1 * D * (LD + 1) * (PD + 1) * (R + 1) * (P + 1) * (L + 1) * (MH + 1)
MH*L*D + MH*R*D*PD + MH*D + L*R*D + L*D*PD + L*D + R*D*PD + R*D + D*PD + D

16 17 18 19 20 22 23 24 26 27 28 31 35 36 37 38 40 41 42 43 44 45 46 48 49 50 52 53 54 55 56 

In [13]:
selection=F*B+A*B*F;
for i = 1:size(uq_expr_list,1)
    res=normal_form(uq_expr_list[i].*selection, gens(GB));
    counter=0;
    for j = 1:size(expr_list,1)
        if (uq_expr_list[i]==expr_list[j])
            counter=counter+1;
            printstyled(indices[j], color=:red);
            print(" ");
        end
    end
    printstyled(" (#", color=:red);
    printstyled(counter, color=:red);
    printstyled(")", color=:red);
    print("\n");
    printstyled(factor(uq_expr_list[i]), color=:green);
    wrtnorm=0;
    if (wrtnorm==1)
        printstyled("=\n", color=:green);
        printstyled(uq_expr_list[i], color=:green);
    end
    print("\n");
    printstyled(res, bold=:true);
    print("\n\n");
end

1  (#1)
1 * D * PD * (LD + 1) * (R + 1) * (P + 1) * (L + 1) * (MH + 1)
0

2 12  (#2)
1 * MH * D * (LD + 1) * (PD + 1) * (R + 1) * (P + 1) * (L + 1)
0

3 4 5 8  (#4)
1 * L * D * PD * (LD + 1) * (R + 1) * (P + 1) * (MH + 1)
MH*L*R*D + MH*R*D + MH*D*PD + L*D*PD

6  (#1)
1 * MH * L * D * (LD + 1) * (PD + 1) * (R + 1) * (P + 1)
0

7  (#1)
1 * MH * L * D * PD * (LD + 1) * (R + 1) * (P + 1)
0

9 10  (#2)
1 * L * D * (LD + 1) * (PD + 1) * (R + 1) * (P + 1) * (MH + 1)
0

11  (#1)
1 * MH * L * D * LD * (PD + 1) * (R + 1) * (P + 1)
0

13 14 15  (#3)
1 * D * (LD + 1) * (PD + 1) * (R + 1) * (P + 1) * (L + 1) * (MH + 1)
0

16 17 18 19 20 22 23 24 26 27 28 31 35 36 37 38 40 41 42 43 44 45 46 48 49 50 52 53 54 55 56 57 58 59 60 61 64 65 66 68 69 70 71 72 73 74 75 76 77 81 82 83 86 87 88 89 91 92 93 94 95 96 97 98 99 101 104 105 106 107 109 110 111 112 114 116 117 118 119 120 122 123 124 130 132 133 134 136 137 138 139 141 142 143 144 145 146 147 148 149 152 154 155 156 157 158 160 162 163 164 165 168 